In [1]:
import pandas as pd
import os

In [18]:
# load the 3 benchmark datasets
datasets = ["libraries", "cedar", "dafnyVMC"]

benchmarks = {}
for dataset in datasets:
    print("Processing", dataset)
    if os.path.exists(f"../results/training_{dataset}.csv") and os.path.exists(
        f"../results/testing_{dataset}.csv"
    ):
        print(f"Training and testing datasets already exist for {dataset}")
        benchmarks[dataset] = {
            "training": pd.read_csv(f"../results/training_{dataset}.csv"),
            "testing": pd.read_csv(f"../results/testing_{dataset}.csv"),
        }
    else:
        df = pd.read_csv(f"../results/non_verified_{dataset}.csv")

        # remove duplicates
        df.drop_duplicates(subset=["Original Method", "Assertion"], inplace=True)

        # rewrite the path of "New Method File" but keep the same file name
        df["New Method File"] = df["New Method File"].apply(
            lambda x: os.path.join("/exp/dafny_repair/results/", os.path.basename(x))
        )

        # split the dataset into training and testing
        training = df.sample(frac=0.5)
        testing = df.drop(training.index)

        # save the datasets
        training.to_csv(f"../results/training_{dataset}.csv", index=False)
        testing.to_csv(f"../results/testing_{dataset}.csv", index=False)
        benchmarks[dataset] = {"training": training, "testing": testing}

Processing libraries
Processing cedar
Processing dafnyVMC


In [19]:
benchmarks["dafnyVMC"]["training"]

,Index,Original File,Original Method,Original Method Time,Original Method Result,Original Method File,Assertion,Time Difference,New Method File,New Method,New Method Time,New Method Result,New Result File
6,14,/exp/test_projects/Dafny-VMC/src/Math/Analysis...,LimitOfInverse,4.634341,Correct,./results/LimitOfInverse_0.txt,"assert ExistsCloseSuffix(sequence, limit, epsi...",NaN,/exp/dafny_repair/results/LimitOfInverse_fix_1...,LimitOfInverse,7.900194,Errors,./results/LimitOfInverse_14.txt
16,28,/exp/test_projects/Dafny-VMC/src/Math/Analysis...,Sqrt2Exists,3.348150,Correct,./results/Sqrt2Exists_0.txt,assert 2.0 in upper;\n,NaN,/exp/dafny_repair/results/Sqrt2Exists_fix_28.dfy,Sqrt2Exists,8.275090,Errors,./results/Sqrt2Exists_28.txt
4,11,/exp/test_projects/Dafny-VMC/src/Math/Analysis...,LimitIsMultiplicative,3.583202,Correct,./results/LimitIsMultiplicative_0.txt,"assert ExistsCloseSuffix(sequence1, limit1, ep...",NaN,/exp/dafny_repair/results/LimitIsMultiplicativ...,LimitIsMultiplicative,6.898129,Errors,./results/LimitIsMultiplicative_11.txt
13,24,/exp/test_projects/Dafny-VMC/src/Math/Analysis...,LimitOfMultiplicationWithZeroSequence,5.956591,Correct,./results/LimitOfMultiplicationWithZeroSequenc...,"assert ExistsCloseSuffix(zeroSeq, 0.0, epsilon...",NaN,/exp/dafny_repair/results/LimitOfMultiplicatio...,LimitOfMultiplicationWithZeroSequence,7.739465,Errors,./results/LimitOfMultiplicationWithZeroSequenc...
10,18,/exp/test_projects/Dafny-VMC/src/Math/Analysis...,BoundOfConvergentSequence,5.700805,Correct,./results/BoundOfConvergentSequence_0.txt,"assert ExistsCloseSuffix(sequence, limit, 1.0);\n",NaN,/exp/dafny_repair/results/BoundOfConvergentSeq...,BoundOfConvergentSequence,8.284380,Errors,./results/BoundOfConvergentSequence_18.txt
5,12,/exp/test_projects/Dafny-VMC/src/Math/Analysis...,LimitIsMultiplicative,3.583202,Correct,./results/LimitIsMultiplicative_0.txt,"assert ExistsCloseSuffix(sequence2, limit2, ep...",NaN,/exp/dafny_repair/results/LimitIsMultiplicativ...,LimitIsMultiplicative,8.953913,Errors,./results/LimitIsMultiplicative_12.txt
29,87,/exp/test_projects/Dafny-VMC/src/Probabilistic...,LiftInEventSpaceToResultEventSpace,5.630720,Correct,./results/LiftInEventSpaceToResultEventSpace_0...,assert Rests(results) == iset{};\n,NaN,/exp/dafny_repair/results/LiftInEventSpaceToRe...,LiftInEventSpaceToResultEventSpace,6.374021,Errors,./results/LiftInEventSpaceToResultEventSpace_8...
21,61,/exp/test_projects/Dafny-VMC/src/Distributions...,UnifCorrectness2,3.374044,Correct,./results/UnifCorrectness2_0.txt,"assert UnifIsCorrect(n, k, m);\n",NaN,/exp/dafny_repair/results/UnifCorrectness2_fix...,UnifCorrectness2,1.625887,Errors,./results/UnifCorrectness2_61.txt
25,72,/exp/test_projects/Dafny-VMC/src/Distributions...,SampleRecursiveHalf,5.064463,Correct,./results/SampleRecursiveHalf_0.txt,assert Independence.IsIndepFunctionCondition(M...,NaN,/exp/dafny_repair/results/SampleRecursiveHalf_...,SampleRecursiveHalf,5.878601,Errors,./results/SampleRecursiveHalf_72.txt
22,62,/exp/test_projects/Dafny-VMC/src/Distributions...,SampleIsMeasurePreserving,4.024397,Correct,./results/SampleIsMeasurePreserving_0.txt,"assert Measures.PreImage(f, e) == preimage';\n",NaN,/exp/dafny_repair/results/SampleIsMeasurePrese...,SampleIsMeasurePreserving,2.817518,Errors,./results/SampleIsMeasurePreserving_62.txt
